In [1]:
from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement
import json
from collections import defaultdict

# Conexión al clúster de Cassandra con los nombres de host de tus nodos
cluster = Cluster(['127.0.0.1'], port=9042)#, auth_provider=auth_provider)

# Crear una sesión
session = cluster.connect()

# Seleccionar el keyspace (si ya tienes uno creado)
session.set_keyspace('videojuego')

## Lectura hall of fame

In [33]:
def get_hall_of_fame(country: str):
    # Establecer consistencia más fuerte
    # Para asegurar la máxima consistencia usamos ALL. Todas las réplicas tendrán que responder a la query
    # session.default_consistency_level = ConsistencyLevel.ALL

    # Query 
    query = """
        SELECT Mazmorra_id, Nombre_mazmorra, Email, Nombre_usuario, Tiempo, Fecha 
        FROM Hall_of_fame 
        WHERE Pais = %s
        ALLOW FILTERING;
    """
    query = SimpleStatement(query,consistency_level = ConsistencyLevel.ALL)
    
    rows = session.execute(query, (country, ))
    
    # Organizar datos en un diccionario agrupado por Mazmorra_id
    dungeon_dict = defaultdict(lambda: {"dungeon_id": 0, "dungeon_name": "", "Top_5": []})

    for row in rows:
        dungeon_id = row.mazmorra_id
        dungeon_dict[dungeon_id]["dungeon_id"] = dungeon_id
        dungeon_dict[dungeon_id]["dungeon_name"] = row.nombre_mazmorra
        dungeon_dict[dungeon_id]["Top_5"].append({
            "email": row.email,
            "user_name": row.nombre_usuario,
            "time_minutes": row.tiempo,
            "date": row.fecha
        })
        
    # Ordenar la lista Top_5 por tiempo en cada mazmorra
    for dungeon in dungeon_dict.values():
      dungeon["Top_5"] = sorted(dungeon["Top_5"], key=lambda x: x["time_minutes"])
    
    result = json.dumps(list(dungeon_dict.values()), indent=4) 
    return result


In [34]:
# Ejemplo de uso
country = "it_IT"
top_players = get_hall_of_fame(country)
print(top_players)


[
    {
        "dungeon_id": 3,
        "dungeon_name": "Wanton, Culverts of the Jealous Thieves",
        "Top_5": [
            {
                "email": "abiagiotti@example.net",
                "user_name": "colucciotaliani",
                "time_minutes": 1.0,
                "date": "2017-12-02 06:47:43"
            },
            {
                "email": "branciforteazeglio@example.org",
                "user_name": "robertoromiti",
                "time_minutes": 1.0,
                "date": "2017-08-02 04:55:47"
            },
            {
                "email": "benvenutoschiavone@example.org",
                "user_name": "lidia16",
                "time_minutes": 1.0,
                "date": "2019-05-07 06:53:03"
            },
            {
                "email": "armaniannetta@example.org",
                "user_name": "fabiavisintini",
                "time_minutes": 1.0,
                "date": "2022-05-01 19:55:21"
            },
            {
               

## Lectura Statistics

In [22]:
def get_user_statistics(user_email: str, dungeon_id: int):
    # Establecer consistencia más fuerte
    
    # Query 
    query = """
        SELECT Tiempo, Fecha 
        FROM Statistic 
        WHERE Email = %s AND Mazmorra_id = %s;
    """
    query = SimpleStatement(query,consistency_level = ConsistencyLevel.ALL)
    
    rows = session.execute(query, (user_email, dungeon_id))
    
    # converitr a lista de diccionarios
    result = [{"time_minute": row.tiempo, "date": row.fecha} for row in rows]
    
    # Convertir a JSON
    return json.dumps(result, indent=4)


In [8]:
# Ejemplo de uso
user_email = 'ansaldo75@example.net'
dungeon_id = 2
statistics_json = get_user_statistics(user_email, dungeon_id)
print(statistics_json)


[
    {
        "time_minute": 0.15000000596046448,
        "date": "2025-03-11 18:13:29"
    },
    {
        "time_minute": 0.4000000059604645,
        "date": "2025-03-11 18:13:21"
    },
    {
        "time_minute": 5.0,
        "date": "2022-05-22 09:38:12"
    },
    {
        "time_minute": 11.0,
        "date": "2022-05-08 00:43:35"
    },
    {
        "time_minute": 12.0,
        "date": "2022-09-16 08:07:53"
    },
    {
        "time_minute": 12.0,
        "date": "2022-06-01 19:39:16"
    },
    {
        "time_minute": 13.0,
        "date": "2022-09-17 15:12:40"
    },
    {
        "time_minute": 14.0,
        "date": "2022-12-26 06:07:28"
    },
    {
        "time_minute": 14.0,
        "date": "2022-08-09 09:11:24"
    },
    {
        "time_minute": 14.0,
        "date": "2022-07-22 14:29:26"
    },
    {
        "time_minute": 14.0,
        "date": "2022-07-22 08:13:55"
    }
]


## Lectura Top horde

In [ ]:
def get_top_horde(country: str, event_id: int, k: int):
    # Reducimos consistencia para asegurar velocidad en las lecturas

    # Query optimizada
    query = """
        SELECT Email, Nombre_usuario, N_killed 
        FROM Top_horde 
        WHERE Evento_id = %s AND Pais = %s 
        LIMIT %s;
    """
    query = SimpleStatement(query,consistency_level = ConsistencyLevel.LOCAL_ONE)
    
    rows = session.execute(query, (event_id, country, k))
    
    # Convertir a lista de diccionarios
    result = [
        {
            "posicion": idx + 1,  # Asignamos un ranking (opcional)
            "user_name": row.nombre_usuario,
            "email": row.email,
            "n_killed": row.n_killed
        }
        for idx, row in enumerate(rows)
    ]

    # Convertir a JSON
    return json.dumps(result, indent=4)


In [19]:
# Ejemplo de uso
country = "ja_JP"
event_id = 2
k = 5
top_horde_json = get_top_horde(country, event_id, k)
print(top_horde_json)


[
    {
        "posicion": 1,
        "user_name": "ltanaka",
        "email": "ymurakami@example.com",
        "n_killed": 30
    },
    {
        "posicion": 2,
        "user_name": "ukobayashi",
        "email": "jyamashita@example.org",
        "n_killed": 22
    },
    {
        "posicion": 3,
        "user_name": "gfukuda",
        "email": "inoueyosuke@example.com",
        "n_killed": 21
    },
    {
        "posicion": 4,
        "user_name": "yasuhiro13",
        "email": "nakagawanaoto@example.org",
        "n_killed": 21
    },
    {
        "posicion": 5,
        "user_name": "rikakato",
        "email": "rika43@example.org",
        "n_killed": 21
    }
]


## Cerramos conexión

In [20]:
# Cerrar la conexión
cluster.shutdown()